# Gaia Events and Observations

The goal of this notebook is to estimate the number of microlensing events we can expect Gaia to discover during the lifetime of a Key Project. 

We draw on two real-world sources.  Firstly, a catalog of microlensing events detected after the fact from complete event lightcurve in the Gaia Data Release 3, published in Wyrzykowski et al. 2022, and secondly the list of microlensing alert candidates announced via the Gaia Alerts system. 

In [1]:
from os import path
import csv
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table, Column
from astropy.io import fits
import numpy as np
import h5py

In [2]:
def read_event_list(event_file_path):
    data = []
    with open(event_file_path, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in reader:
            s = SkyCoord(row[1], row[2], frame='icrs', unit=(u.hourangle, u.deg))
            data.append(row + [s])
    data = np.array(data)

    event_table = Table(
                        [Column(data=data[:,0], name='Event'),
                        Column(data=data[:,1], name='RA'),
                        Column(data=data[:,2], name='Dec'),
                        Column(data=data[:,3], name='Coordinate')]
                        )

    return event_table

In [3]:
def read_gaia_microlensing_alerts_list(alerts_file_path):
    data = []
    with open(alerts_file_path, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in reader:
            if '#' not in row[0]:
                print(row)
                if 'microlensing' in str(row[9]).lower():
                    s = SkyCoord(row[3], row[4], frame='icrs', unit=(u.deg, u.deg))
                    data.append([row[0], row[3], row[4], s])
    data = np.array(data)

    event_table = Table(
                        [Column(data=data[:,0], name='Event'),
                        Column(data=data[:,1], name='RA'),
                        Column(data=data[:,2], name='Dec'),
                        Column(data=data[:,3], name='Coordinate')]
                        )

    return event_table

Let's read in both lists and compare the set of events discovered

In [4]:
event_file_path = 'gaia_event_list.csv'
alerts_file_path = 'gaia_alerts_data.csv'

gaia_events = read_event_list(event_file_path)
gaia_alerts = read_gaia_microlensing_alerts_list(alerts_file_path)

['Gaia22epg', '2022-10-27 03:42:17', '19.96019', '-53.95500', '18.74', '19.83', '0.40', '"unknown"', '2022-11-16 10:15:54', '"brightening in known AGN"', 'AT2022aalv']
['Gaia22epf', '2022-10-26 22:50:39', '274.77416', '-28.17701', '18.30', '19.55', '0.05', '"unknown"', '2022-11-16 10:15:44', '"gal.plane source candidate microlensing event rises by 1.3 mag"', 'AT2022aalu']
['Gaia22epe', '2022-10-26 12:36:09', '273.40098', '-29.60688', '18.13', '19.30', '0.04', '"unknown"', '2022-11-16 10:15:38', '"gal.plane source candidate microlensing event rises by 1.2 mag"', 'AT2022aalt']
['Gaia22epd', '2022-10-26 16:55:07', '271.10083', '-24.27362', '17.02', '18.34', '0.36', '"unknown"', '2022-11-16 10:15:07', '"brightening in candidate YSO"', 'AT2022aals']
['Gaia22epc', '2022-10-26 15:45:22', '14.69411', '-56.98653', '15.91', '16.86', '0.18', '"unknown"', '2022-11-16 10:14:52', '"brightening in known QSO"', 'AT2022aalr']
['Gaia22epb', '2022-10-26 16:51:48', '272.94736', '-27.39745', '17.66', '18.1

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
print('Total number of lensing events in Gaia DR3: '+str(len(gaia_events)))
print('Total number of microlensing alerts: '+str(len(gaia_alerts)))

Total number of lensing events in Gaia DR3: 445
Total number of microlensing alerts: 389


In [6]:
alerted_events = []
only_alerted_events = []
for event in gaia_alerts:
    if event['Event'] in gaia_events['Event'].data:
        alerted_events.append(event['Event'])
    else:
        only_alerted_events.append(event['Event'])
    
print(str(len(alerted_events))+' events found in DR 3 were alerted in real-time')
print(str(len(only_alerted_events))+' events were only alerted in real-time')

discovered_events = []
for event in gaia_events:
    if event['Event'] not in gaia_alerts['Event'].data:
        discovered_events.append(event['Event'])

print(str(len(discovered_events))+' events were only discovered in the offline analysis')

122 events found in DR 3 were alerted in real-time
267 events were only alerted in real-time
323 events were only discovered in the offline analysis


In [7]:
def count_events_per_year(event_set):
    events_per_year = {}
    for event in event_set:
        year = '20' + str(event['Event'])[4:6]
        if year not in events_per_year.keys():
            events_per_year[year] = 1
        else:
            events_per_year[year] += 1

    for key, value in events_per_year.items():
        print(str(key)+': '+str(value)+' events')

Now let's take a look at the expected rate of discoveries, bearing in mind that the survey and the alert system took a while to ramp up to the best rate of discoveries. 

In [8]:
print('Events per year discovered in DR 3 data:')
count_events_per_year(gaia_events)

print('\nAlerts per year:')
count_events_per_year(gaia_alerts)

Events per year discovered in DR 3 data:
2016: 3 events
2017: 26 events
2018: 72 events
2019: 106 events
2020: 118 events
2021: 120 events

Alerts per year:
2022: 172 events
2021: 75 events
2020: 87 events
2019: 42 events
2018: 5 events
2017: 4 events
2016: 3 events
2015: 1 events


Based on the categorized list of events from 2021, 11 binary lensing events were detected, and 9 binary events were detected in 2020.

## Expected number of events within a Key Project

From this it is clear that the rate of discovery is quite variable, but approximately 80-120 events per year is a reasonable expectation.  Based on this, we adopt a nominal rate of 100 events/year. 

We note that within the lifetime of a Key Project, the Gaia Mission is expected to continue operations between 2023-2025.  We therefore nominally anticipate ~200 events discovered by Gaia during the Key Project, of which we expect ~10% to be binary events.  In addition to this, there is a catalog of ~50 ongoing events alerted in 2021-2022 with tE betwee 100 - 350d, which we will monitor long term. 

We also note that the Gaia alerts system is being maintained on a best-efforts basis for as long as possible.  In the event that no further Gaia alerts are produced, we will focus on characterising the events detected up until the system halted, and from Gaia DR.  

Based on the kp_simulator simulation, LCO would be able to observe 75.5% of all Gaia discovered events around the event peak, and approximately 7.9% of the events are expected to be candidate BH lensing events. 
Long duration microlensing events are easy to detect from two-filter survey data.  They can only really be confused with long period variables such as Be stars and Miras, and their color signature can be used to eliminate false positives relatively easily.  

In [78]:
total_gaia_events = 200
n_years = 2
frac_bh_events = 0.079
frac_observable = 0.75
n_bh_events_obs = int(round((frac_bh_events * total_gaia_events * frac_observable),0))
print('We therefore expect ~'+str(round(n_bh_events_obs,0))\
      +' long-duration events to be confirmed during the lifetime of the Key Project, '
      +str(n_bh_events_obs/n_years)+' per year')

We therefore expect ~12 long-duration events to be confirmed during the lifetime of the Key Project, 6.0 per year


Based on event rates from previous years we also anticiate the following stellar lensing events, which are sensitive to the signatures of planetary mass companions also.  We propose to provide high-cadence photometry of these events around the peak as well, in order to characterize the stellar binaries and check for planetary companions.  

In [79]:
n_years = 2
frac_stellar_events = 0.921
n_stellar_events_obs = int(round((frac_stellar_events * total_gaia_events * frac_observable),0))
print('We expect ~'+str(round(n_stellar_events_obs,0))\
      +' stellar events to be observed during the lifetime of the Key Project, '\
      +str(n_stellar_events_obs/n_years)+' per year')


We expect ~138 stellar events to be observed during the lifetime of the Key Project, 69.0 per year


Inevitably when observing transient events, we cannot know in advance which events will exhibit binary anomalies.  Indeed, without follow-up observations, most Gaia stellar binary anomalies will be missed entirely.  In order to ensure that the binary events are properly identified, we propose to monitor at moderate cadence all stellar events which are predicted to reach high-magnification.  

These observations will enable us to identify anomalous events in progress.  Anomalous features and caustic crossings will be monitored at very high cadence (every 15mins) for 12hrs per anomalous event.   If an event does not exhibit anomalous features, it will continue to be monitored at moderate cadence until it becomes to faint for LCO to observe. 

In [109]:
frac_binary_events = 0.1
n_binary_events_obs = int(round((frac_binary_events * frac_stellar_events * total_gaia_events * frac_observable),0))
print('We expect ~'+str(round(n_binary_events_obs,0))\
      +' binary events to be observable during the lifetime of the Key Project, '
     +str(n_binary_events_obs/n_years)+' per year')


We expect ~14 binary events to be observable during the lifetime of the Key Project, 7.0 per year


## Observing Time Required

We can use the simulations of the Key Project observing strategy to estimate the total time required to observe Gaia events. The kp_simulator notebook generated a simulated set of 1000 events, and stored the simulated data products, which we read in here.  

In [57]:
def read_sim_event_table(file_path):
    hdul = fits.open(file_path)
    cols = hdul[1].columns
    data = hdul[1].data
    column_list = []
    for col in cols:
        if col.name == 'EventID':
            dtype = 'str'
        elif col.name in ['HEALpixel', 'nvisits']:
            dtype = 'int'
        else:
            dtype = 'float'
        column_list.append(Column(name=col.name, data=data[col.name], dtype=dtype))
    
    return Table(column_list)

In [58]:
def read_lco_sim_data(file_path, n_events):
    f = h5py.File(file_path, "r")
    dataset = {}
    for eventid in f.keys():
        dataset[eventid] = f[eventid][:]
    
    return dataset

In [59]:
sim_lco_lc_file = '../simulated_lco_lightcurves_gaia_events.hdf5'
dataset = read_lco_sim_data(sim_lco_lc_file, 1000)

In [60]:
sim_data_file = '../sim_gaia_events_table.fits'
events_table = read_sim_event_table(sim_data_file)

In [61]:
events_table

EventID,HEALpixel,RA_deg,Dec_deg,baseline_mag,t0,u0,tE,rho,piEN,piEE,nvisits_gaia,binary_lens
str14,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
GaiaSim00001,16462,290.390625,19.471220016479492,19.489999771118164,2460580.25,0.24013586342334747,48.940547943115234,0.03383931145071983,-0.17942728102207184,0.03151465952396393,218.0,0.0
GaiaSim00002,38205,265.78125,-33.51005554199219,17.989999771118164,2460702.5,0.37446874380111694,20.75156593322754,0.001647921628318727,-0.7691926956176758,0.013071971014142036,119.0,1.0
GaiaSim00003,14927,291.796875,23.317955017089844,18.489999771118164,2460877.0,0.41706404089927673,42.575950622558594,0.0011833814205601811,-0.29106974601745605,-0.19437921047210693,417.0,1.0
GaiaSim00004,42236,250.9322052001953,-45.783966064453125,17.489999771118164,2460278.0,0.48853614926338196,9.209157943725586,0.008074616082012653,0.4106876850128174,-0.02565484680235386,191.0,0.0
GaiaSim00005,14417,294.609375,24.624317169189453,11.989999771118164,2460229.5,0.25978824496269226,11.585479736328125,0.033562447875738144,1.4216102361679077,0.006296116393059492,344.0,0.0
GaiaSim00006,14925,288.984375,23.317955017089844,17.989999771118164,2460394.25,-0.18596400320529938,5.129355430603027,0.006933162454515696,0.2554420828819275,-0.016762293875217438,445.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
GaiaSim00994,34366,267.890625,-23.317955017089844,18.989999771118164,2460480.75,-0.05427982285618782,21.355382919311523,0.016667209565639496,-0.17443227767944336,0.2059374749660492,143.0,0.0
GaiaSim00995,25411,274.21875,-1.7907832860946655,19.489999771118164,2460297.5,0.008039955981075764,5.634570121765137,0.009821082465350628,-0.11065229028463364,0.21534553170204163,115.0,0.0


Selecting the long timescale events with tE > 100d, we can use the simulated LCO lightcurves produced by the Key Project observing strategy, to estimate the typical number of visits to target that we can expect.  We can also use the same data to estimate the total exposure time per target, since the simulation includes target brightness as a function of time.  In order to estimate realistic exposure times on LCO 1m telescopes for targets at these magnitudes, we use the following empirical function, which was derived from previous LCO 1m photometry. 

In [62]:
def ETC(magin):
    """
    This function calculates the required exposure time
    for a given iband magnitude (e.g. OGLE I which also
    roughly matches SDSS i) based on a fit to the empiric
    LCO photometry.  Adapted from code by Markus Hundertmark.
    """
    
    mag = np.zeros(len(magin))
    mag[:] = magin
    
    # Cap the exposures at the bright end to minimise scintillation noise (min 5s)
    idx = np.where(magin < 14.7)
    mag[idx] = 14.7

    lrms = 0.14075464 * mag * mag - 4.00137342 * mag + 24.17513298
    snr = 1.0 / np.exp(lrms)

    # target 4% -> snr 25
    exptime = np.round((25. / snr)**2 * 300.,1)
    
    #Avoid exposures longer than 5min:
    idx = np.where(exptime > 300.0)
    exptime[idx] = 300.0

    return  exptime

In addition to the open-shutter time required during each visit to target, we need to account for telescope slew, instrument readout etc.  Based on the LCO instrumentation website (https://lco.global/observatory/instruments/), the overheads per frame for the LCO 1m Sinistro cameras is 28s.  So we add this to each exposure. 

In [63]:
def estimate_obs_time(event_set, events_table, dataset):
    
    lco_obs_time = []
    lco_nvisits = []
    
    for ievent in event_set:
        event_name = events_table[ievent]['EventID']
        lco_lc = dataset[event_name]
        lco_nvisits.append(len(lco_lc))

        # Calculate the exposure time per visit based on the target magnitude at the time, then 
        # add the instrumental overheads
        if len(lco_lc) > 0:
            lco_exposures = ETC(lco_lc[:,1])
            lco_exposures += 28.0
        else:
            lco_exposures = np.zeros(1)
        
        lco_obs_time.append(lco_exposures.sum())
        
    lco_nvisits = np.array(lco_nvisits)
    lco_obs_time = np.array(lco_obs_time)/3600.0

    return lco_nvisits, lco_obs_time


In [85]:
# Get the table indices of long-duration events
bh_events = np.where(events_table['tE'] > 100.0)[0]

# Estimate the number of visits per target and observing time required:
(lco_nvisits, lco_obs_time) = estimate_obs_time(bh_events, events_table, dataset)

time_per_event = np.median(lco_obs_time)

print('Median number of LCO visits per event: '+str(np.median(lco_nvisits)))
print('Median total LCO exposure time per event: '+str(time_per_event)+'hrs')

Median number of LCO visits per event: 40.0
Median total LCO exposure time per event: 3.059555555555556hrs


In [86]:
total_time_bh_events = n_bh_events_obs * time_per_event
print('The expected sample of '+str(n_bh_events_obs)+' long-duration events is therefore estimated to required a total of '+str(round(total_time_bh_events,0))+'hrs')
       

The expected sample of 12 long-duration events is therefore estimated to required a total of 37.0hrs


To estimate the time required for stellar binary events, we adopt a similar approach, but select the correspondingly shorter timescale events from the simulated sample.

In [111]:
# Get the table indices of single-lens stellar events
stellar_lenses = np.where(events_table['tE'] < 100.0)[0]
single_lenses = np.where(events_table['binary_lens'] == 0)[0]
single_stellar_lenses = list(set(stellar_lenses).intersection(set(single_lenses)))

# Estimate the number of visits per target and observing time required:
(lco_nvisits, lco_obs_time) = estimate_obs_time(single_stellar_lenses, events_table, dataset)

time_per_event = np.median(lco_obs_time)

print('Median number of LCO visits per event: '+str(np.median(lco_nvisits))+' for '+str(len(single_stellar_lenses))+' events')
print('Median total LCO exposure time per event: '+str(time_per_event)+'hrs')

Median number of LCO visits per event: 144.0 for 829 events
Median total LCO exposure time per event: 10.942444444444446hrs


In [112]:
total_time_stellar_events = n_stellar_events_obs * time_per_event
print('The expected sample of '+str(n_stellar_events_obs)+' single-lens stellar events is estimated to required a total of '+str(round(total_time_stellar_events,0))+'hrs')


The expected sample of 138 single-lens stellar events is estimated to required a total of 1510.0hrs


Binary lens events are allocated an additional 12hrs of high-cadence (every 15min) observations on top of the regular monitoring.

In [117]:
add_time_binary_events = n_binary_events_obs * 12.0
print('The '+str(n_binary_events_obs)+' binary events are estimated to require an additional '+str(round(add_time_binary_events,0))+'hrs')



The 14 binary events are estimated to require an additional 168.0hrs


In [119]:
total_time = total_time_bh_events + total_time_stellar_events + add_time_binary_events
total_time_per_year = total_time / 2.0
print('In total, LCO observations of Gaia targets would be expected to require: '+str(total_time)+'hrs')
print('or a total of : '+str(total_time_per_year)+'hrs per year')

In total, LCO observations of Gaia targets would be expected to require: 1714.7720000000002hrs
or a total of : 857.3860000000001hrs per year


## Observations from the 2m Network and Rapid Response Observations

The 2m network offers the MusCAT multi-channel imagers and the FLOYDS spectrographs.  

While only a few targets will be accessible to FLOYDS (r<15mag), the spectra are extremely useful to constrain the source star parameters for all categories of microlensing event.  In order to distinguish the source star spectrum from neighboring objects, spectra at multiple epochs with different lensing magnifications are required, though not at the high cadence required for photometry.  We propose to obtain 3 spectra of selected targets that reach magnitudes brighter than 15mag (where FLOYDS can deliver S/N=100 in 1hr of exposure).  

We can estimate the potential number of FLOYDS targets using the simulated LCO lightcurve dataset, and identifing the fraction of targets likely to exceed 15mag.  

In [87]:
sim_survey_lc_file = '../simulated_gaia_lightcurves.hdf5'
survey_dataset = read_lco_sim_data(sim_survey_lc_file, 1000)

In [97]:
bright_events = []
n_survey_lc = 0
for ievent in range(0,len(survey_dataset),1):
    event_name = events_table[ievent]['EventID']
    
    survey_lc = dataset[event_name]
    if len(survey_lc) > 0:
        n_survey_lc += 1
        idx = np.where(survey_lc[:,1] < 15.0)[0]
        if len(idx) > 0:
            bright_events.append(ievent)

frac_bright_events = float(len(bright_events))/float(len(events_table))
print('Fraction of events accessible to FLOYDS = '+str(frac_bright_events))
print('Number of full survey lightcurves '+str(n_survey_lc))

Fraction of events accessible to FLOYDS = 0.095
Number of full survey lightcurves 765


In [98]:
n_targets = n_bh_events_obs + n_stellar_events_obs + n_binary_events_obs
n_FLOYDS_targets = int(round((n_targets * frac_bright_events),0))
print('Number of Gaia targets likely to be accessible to FLOYDS = '+str(n_FLOYDS_targets))

Number of Gaia targets likely to be accessible to FLOYDS = 16


Since there is a FLOYDS spectrograph in both hemispheres, the 2m network can observe any target while it is visible to our target surveys.  Of the stellar and BH targets we expect per year we expect to be able to observe ~ 2 with FLOYDS.  
Each FLOYDS observation will consist of a 1hr science exposure plus the LCO-recommended set of calibration exposures: 2x60s arc frames with the HgAr and Zn lamps for wavelength calibration and 2x70s flat field exposures with the Tungsten-Halogen + Xenon lamp, since we will use the 1.2" slit due to the crowding in this field.  Calibration frames will be requested both before and after each science spectrum.  

Following the overhead calculation recommended on the FLOYDS website for a 1hr science spectrum:
Initial setup and telescope slew is 120 s. 
Target acquisition is 90 s. 
Lamp flat exposures include an overhead of 60 s.
Arc lamp exposures include an overhead of 100 s. 
All configuration changes (from flat to arc to science spectrum) require a 16 s software setup overhead. 
Readout time is 26 s. 

In [99]:
# All times in s
tel_slew = 120.0
target_acquisition = 90.0
lamp_exp_overhead = 60.0
lamp_exp = 70.0
arc_exp_overhead = 100.0
arc_exp = 60.0
software = 16.0
readout = 26.0
science_exp = 3600.0

In [100]:
time_per_FLOYDS_visit = tel_slew + target_acquisition \
                    + lamp_exp_overhead + software + lamp_exp + readout \
                    + arc_exp_overhead + software + arc_exp + readout \
                    + software + science_exp + readout \
                    + lamp_exp_overhead + software + lamp_exp + readout \
                    + arc_exp_overhead + software + arc_exp + readout
time_per_FLOYDS_visit /= 60.0

print('Time required per FLOYDS visit to 1 target = '+str(time_per_FLOYDS_visit)+'min')

Time required per FLOYDS visit to 1 target = 76.66666666666667min


Taking 3 visits per FLOYDS target therefore requires a total request of (split 50:50 North/South):

In [101]:
total_FLOYDS_time = n_FLOYDS_targets * time_per_FLOYDS_visit * 3.0
total_FLOYDS_time /= 60.0

print('Total FLOYDS time per target = '+str(time_per_FLOYDS_visit * 3.0/60.0)+'hrs')
print('Total time = '+str(total_FLOYDS_time)+'hrs')

Total FLOYDS time per target = 3.8333333333333335hrs
Total time = 61.333333333333336hrs


The MuSCAT four-channel imagers are particularly important for binary lens events, as they can deliver high-cadence multi-color observations of caustic crossings, short-lived, sharp discontinuities in the lightcurves that mark the entrance and exits of caustic crossings lasting ~ 6hrs.  It is extremely important to monitor their photometry continuously for that period in order to constrain the morphology of the lightcurve, which in turn constrains the models of the caustic structure and lens-source relative trajectory.  

While almost all of the observations from this program can be made in queue-scheduled mode, the short timescale and transient nature of these features require rapid response time to ensure that we can observe as soon as they are detected by our real-time modeling system.  They also justify observing from every site in the network to ensure as continuous coverage as possible.  For this reason, we request the following time on MuSCAT and some 1m time is allocated in Rapid Response mode.

Caustic crossings are only likely in the stellar binary events we expect from Gaia.  Adopting an average duration of 6hrs per caustic entrance and exit (i.e. 12hrs per target), we request in Rapid Response mode:

In [106]:
time_caustic = 6.0 # hrs, with two caustic crossings per event
time_rr_stellar_events = n_binary_events_obs * 2.0 * time_caustic
print('Rapid Response Time requested for stellar binary caustic crossings = '+str(time_rr_stellar_events)+'hrs')

Rapid Response Time requested for stellar binary caustic crossings = 168.0hrs


Gaia targets are accessible to FTN/MusCAT or from FTS/Spectral (and the new MusCAT once available) for up to 8hrs per day, or a day fraction of:

In [107]:
vis_fraction = 8.0 / 24.0
vis_fraction

0.3333333333333333

We therefore request that this fraction of the require rapid response time from the 2m imagers, with the remaining rapid response time coming from the 1m/Sinistro network. 

In [108]:
time_2m_imagers = time_rr_stellar_events * vis_fraction
print('Time on 2m imagers in Rapid Response mode = '+str(time_2m_imagers))

Time on 2m imagers in Rapid Response mode = 56.0
